```
Note pour prochaine fois :

J'ai reussi à charger toutes les images dans une liste et je voulais ensuite concatener les 60 images en une seule, mais le fait qu'elles aient des résolutions différentes me bloque.
Faudrait donc ré-échantillonné avant ou alors arreter les bétises et faire comme le prof à fait.
```

# Importation

In [1]:
import geopandas as gpd
from osgeo import gdal
import os
import numpy as np
import sys
sys.path.append('/home/onyxia/work/libsigma')
import read_and_write as rw

# Raster 60 bandes

In [2]:
#  Définition des paramètres 

input_raster_dir = "/home/onyxia/work/data/images"
L_images = sorted(os.listdir(input_raster_dir))
L_images.remove('.keep')
ref_raster_path = "/home/onyxia/work/data/images/SENTINEL2B_20220125-105852-948_L2A_T31TCJ_C_V3-0_SRE_B2.tif"
shapefile_path = "/home/onyxia/work/data/project/emprise_etude.shp"
mask_foret_path = "/home/onyxia/Projet_Teledec/results/data/img_pretraitees/masque_foret.tif"
output_dir = "/home/onyxia/work"

os.makedirs(output_dir, exist_ok=True)

In [ ]:
# Construction array

x,y = rw.get_image_dimension(rw.open_image(ref_raster_path))[:2]
bandes = 60
array_tot = np.zeros((x,y,bandes))

L_array = []
for i,img in enumerate(L_images):
    path = os.path.join(input_raster_dir,img) 
    array = rw.load_img_as_array(path)
    L_array.append(array)

60

In [ ]:
driver = gdal.GetDriverByName('GTiff')
out_filename = 'Serie_temp_S2_allbands.tif'
nb_col = 4
nb_ligne = 4
nb_band = 2
gdal_dtype = gdal.GDT_Int32
output_data_set = driver.Create(out_filename, nb_col, nb_ligne, nb_band, gdal_dtype)


# Clip Raster

In [ ]:
#Façon potentiel pour clip un raster grâce à l'emprise
#Pour l'instant ne pas faire tourner

# Charger le vecteur avec Geopandas
gdf = gpd.read_file("/home/onyxia/work/data/project/emprise_etude.shp").to_crs("EPSG:2154")

# Extraire le GeoJSON sous forme de string
geojson_str = gdf.to_json()

# Appliquer le clip avec GDAL
resolution = 10  # Résolution (10 m)
gdal.Warp(
    "/home/onyxia/work/test.tif",     #Chemin fichier en sortie
    "/home/onyxia/work/data/images/SENTINEL2B_20220125-105852-948_L2A_T31TCJ_C_V3-0_SRE_B2.tif", #Chemin fichier en entrée
    format="GTiff",
    cutlineDSName=geojson_str,  # Passer directement le GeoJSON
    cropToCutline=True,
    dstSRS="EPSG:2154",
    xRes=resolution,
    yRes=resolution,
    dstNodata=0
)

/opt/conda/lib/python3.12/site-packages/osgeo/gdal.py:311: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7f4bf7603450> >